In [23]:
# Dependencies


import numpy as np
import functools as ft
import networkx as nx
from sklearn.base import BaseEstimator, ClassifierMixin
from datasets import load_dataset
from sklearn.model_selection import cross_val_score

In [24]:
# Basic arithemitic operations


def hdv(d):
    return np.random.choice([-1, 1], d)


def bind(xs):
    return ft.reduce(lambda x, y: x * y, xs)


def bundle(xs):
    return ft.reduce(lambda x, y: x + y, xs)


def similarity(A, B):
    return np.dot(A, B) / len(A)


def cosine_similarity(A, B):
    dot_product = np.dot(A, B)
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)

    if norm_A == 0 or norm_B == 0:
        return 0

    return dot_product / (norm_A * norm_B)

In [25]:
# Memory


class ItemMemory:
    def __init__(self, vectors=[]):
        self.vectors = vectors

    def addVector(self, label, V):
        self.vectors.append((label, V))

    def cleanup(self, V):
        return max(self.vectors, key=lambda x: cosine_similarity(V, x[1]))

In [26]:
# Stochastic arithemtic operations


def weightedAverage(A, B, p, q):
    return np.fromiter(
        map(lambda t: np.random.choice([t[0], t[1]], p=[p, q]), zip(A, B)),
        dtype=np.int_,
    )


def hdvA(B, a):
    return weightedAverage(B, -B, (a + 1) / 2, (1 - a) / 2)

In [27]:
# Basic tests

A, B = hdv(1000), hdv(1000)
C = weightedAverage(A, B, 0.90, 0.1)
# print(A)
# print(B)
# print(C)
# print(cosine_similarity(A, C))
# print(cosine_similarity(B, C))

Ba = hdvA(B, 0.5)
print(cosine_similarity(B, Ba))

0.508


In [32]:
def processDataset(dataset):
    graphs = []
    labels = []

    for graph in dataset:
        G = nx.Graph()
        G.add_edges_from(zip(graph["edge_index"][0], graph["edge_index"][1]))
        graphs.append(G)
        labels.append(graph["y"][0])

    return (graphs, labels)


def transform(X, alpha):
    graphs = []
    for graph in X:
        gpr = nx.pagerank(graph, alpha)
        H = nx.relabel_nodes(graph, gpr)
        graphs.append(H)
    return graphs


def encodeGraph(graph, vertices, base):
    for node in graph.nodes:
        if node not in vertices:
            # print(node)
            vertices[node] = hdvA(base, node)
            # vertices[node] = hdv(10000)

    Edges = []

    for edge in graph.edges:
        v1 = vertices[edge[0]]
        v2 = vertices[edge[1]]
        E = bind([v1, v2])
        Edges.append(E)

    Graph = bundle(Edges)

    return Graph

In [29]:
class GraphHD(BaseEstimator, ClassifierMixin):
    def __init__(self, alpha=0.45, dimensions=10000, step=20):
        self.alpha = alpha
        self.step = step
        self.dimensions = dimensions
        self.base = hdv(dimensions)

    def fit(self, X, y):
        self.memory = ItemMemory([])
        self.vertices = dict()
        self.labels = list(set(y))
        dictLabels = dict()

        graphs = transform(X, self.alpha)

        for label in self.labels:
            dictLabels[label] = []

        for i in range(len(graphs)):
            Graph = encodeGraph(graphs[i], self.vertices, self.base)
            dictLabels[y[i]].append(Graph)

        for key, value in dictLabels.items():
            for i in range(0, len(value), self.step):
                H = bundle(value[i : i + self.step])
                self.memory.addVector(str(key), H)

        return self

    def predict(self, X):
        p = []
        s = []

        graphs = transform(X, self.alpha)

        for testGraph in graphs:
            queryVector = encodeGraph(testGraph, self.vertices, self.base)
            cleanVector = self.memory.cleanup(queryVector)

            p.append(int(cleanVector[0]))
            # s.append(cosine_similarity(queryVector, cleanVector[1]))

        # print("%.5f" % round(np.mean(s), 5), "0:", p.count(0), "1:", p.count(1))
        return p

In [30]:
MUTAG = load_dataset("graphs-datasets/MUTAG")
# PROTEINS = load_dataset("graphs-datasets/PROTEINS")
(graphs, labels) = processDataset(MUTAG["train"])

In [34]:
FOLDS, REPS = 10, 1

# MUTAG
ALPHA, DIMENSIONS, STEP = 0.65, 10000, 20

# PROTEINS
# ALPHA, DIMENSIONS, STEP = 0.65, 10000, 4


def main():
    clf = GraphHD(ALPHA, DIMENSIONS, STEP)
    sum = 0
    for i in range(REPS):
        scores = cross_val_score(clf, graphs, labels, n_jobs=-1, cv=FOLDS, verbose=3)
        sum += scores.mean()
        print(i, "->", "%.5f" % scores.mean())
        del scores

    print("S => %.5f" % (sum / REPS))


main()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.


[CV] END ................................ score: (test=0.368) total time= 4.2min
[CV] END ................................ score: (test=0.737) total time= 4.3min
[CV] END ................................ score: (test=0.737) total time= 4.3min


[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:  4.3min remaining: 10.1min


[CV] END ................................ score: (test=0.947) total time= 4.3min
[CV] END ................................ score: (test=0.737) total time= 4.4min
[CV] END ................................ score: (test=0.444) total time= 4.4min
[CV] END ................................ score: (test=0.667) total time= 4.4min


[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:  4.4min remaining:  1.9min


[CV] END ................................ score: (test=0.842) total time= 4.4min
[CV] END ................................ score: (test=0.737) total time= 4.4min
[CV] END ................................ score: (test=0.579) total time= 4.6min
0 -> 0.67953
S => 0.67953


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  4.6min finished
